In [ ]:
import cv2
import math
import argparse
import mysql.connector
import face_recognition

# Database connection
mydb = mysql.connector.connect(host="localhost", user="root", password="Gowtham$3", database="presenting")
cursor = mydb.cursor()

# Function to add column if it doesn't exist
def add_column_if_not_exists(cursor, table_name, column_name, column_definition):
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM information_schema.COLUMNS 
        WHERE TABLE_NAME = '{table_name}' 
        AND COLUMN_NAME = '{column_name}'
    """)
    if cursor.fetchone()[0] == 0:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_definition}")

# Ensure the FaceCount column exists
add_column_if_not_exists(cursor, 'collecting_data', 'FaceCount', 'INT DEFAULT 1')

def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    faceBoxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceBoxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)), 8)
    return frameOpencvDnn, faceBoxes

parser = argparse.ArgumentParser()
parser.add_argument('--image')
args, unknowns = parser.parse_known_args()

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

video_source = args.image if args.image else 0
video = cv2.VideoCapture(video_source)
padding = 20

# List to store face encodings of detected faces
known_face_encodings = []

while cv2.waitKey(1) < 0:
    hasFrame, frame = video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg, faceBoxes = highlightFace(faceNet, frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face = frame[max(0, faceBox[1] - padding):min(faceBox[3] + padding, frame.shape[0] - 1),
                     max(0, faceBox[0] - padding):min(faceBox[2] + padding, frame.shape[1] - 1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print(f'Gender: {gender}')

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print(f'Age: {age[1:-1]} years')

        # Get face encoding
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_frame, [(faceBox[1], faceBox[2], faceBox[3], faceBox[0])])

        if face_encodings:
            face_encoding = face_encodings[0]

            # Check if this face is already in known_face_encodings
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)

            if not any(matches):
                # It's a new face
                known_face_encodings.append(face_encoding)

                # Insert the new detection into the database
                add_data = ("INSERT INTO collecting_data (Gender, Age, FaceCount) VALUES (%s, %s, %s)")
                data = (gender, age[1:-1], 1)
                cursor.execute(add_data, data)
            else:
                # Existing face detected, update the FaceCount
                update_data = ("UPDATE collecting_data SET FaceCount = FaceCount + 1 WHERE Gender = %s AND Age = %s")
                data = (gender, age[1:-1])
                cursor.execute(update_data, data)

            mydb.commit()

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)

cursor.close()
mydb.close()


In [ ]:
import cv2
import math
import argparse
import mysql.connector
import face_recognition

# Database connection
mydb = mysql.connector.connect(host="localhost", user="root", password="Gowtham$3", database="presenting")
cursor = mydb.cursor()

# Function to add column if it doesn't exist
def add_column_if_not_exists(cursor, table_name, column_name, column_definition):
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM information_schema.COLUMNS 
        WHERE TABLE_NAME = '{table_name}' 
        AND COLUMN_NAME = '{column_name}'
    """)
    if cursor.fetchone()[0] == 0:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_definition}")

# Ensure the FaceCount column exists
add_column_if_not_exists(cursor, 'collecting_data', 'FaceCount', 'INT DEFAULT 1')

def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    faceBoxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceBoxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)), 8)
    return frameOpencvDnn, faceBoxes

parser = argparse.ArgumentParser()
parser.add_argument('--image')
args, unknowns = parser.parse_known_args()

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

video_source = args.image if args.image else 0
video = cv2.VideoCapture(video_source)
padding = 20

# List to store face encodings of detected faces
known_face_encodings = []

while cv2.waitKey(1) < 0:
    hasFrame, frame = video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg, faceBoxes = highlightFace(faceNet, frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face = frame[max(0, faceBox[1] - padding):min(faceBox[3] + padding, frame.shape[0] - 1),
                     max(0, faceBox[0] - padding):min(faceBox[2] + padding, frame.shape[1] - 1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print(f'Gender: {gender}')

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print(f'Age: {age[1:-1]} years')

        # Get face encoding
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_frame, [(faceBox[1], faceBox[2], faceBox[3], faceBox[0])])

        if face_encodings:
            face_encoding = face_encodings[0]

            # Check if this face is already in known_face_encodings
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)

            if not any(matches):
                # It's a new face
                known_face_encodings.append(face_encoding)

                # Insert the new detection into the database
                add_data = ("INSERT INTO collecting_data (Gender, Age, FaceCount) VALUES (%s, %s, %s)")
                data = (gender, age[1:-1], 1)
                cursor.execute(add_data, data)
            else:
                # Existing face detected, update the FaceCount
                update_data = ("UPDATE collecting_data SET FaceCount = FaceCount + 1 WHERE Gender = %s AND Age = %s")
                data = (gender, age[1:-1])
                cursor.execute(update_data, data)

            mydb.commit()

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)

cursor.close()
mydb.close()


No face detected
No face detected
No face detected
No face detected
Gender: Male
Age: 8-12 years
Gender: Male
Age: 8-12 years
Gender: Male
Age: 4-6 years
Gender: Male
Age: 4-6 years
Gender: Male
Age: 4-6 years
Gender: Male
Age: 8-12 years
Gender: Male
Age: 48-53 years
Gender: Male
Age: 25-32 years
Gender: Male
Age: 25-32 years
Gender: Male
Age: 48-53 years
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
Gender: Female
Age: 25-32 years
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
Gender: Male
Age: 48-53 years
Gender: Male
Age: 48-53 years
Gender: Male
Age: 25-32 years
No face detected
No face detected
No face detected
No face detected
Gender: Female
Age: 25-32 years
No face detected
Gender: Female
Age: 25-32 years
Gender: Male
Age: 48-53 years
Gender: Male
Age: 25-32 years
Gender: Male
Age: 25-32 years
Gender: 